In [16]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
import numba
from numba import jit

from scipy.io import wavfile
from IPython.display import Audio 

## Initialisation

In [145]:
Nx = 101 ## points for the length of the string
L =0.7
dt = 5e-6
dx = L/(Nx-1)

l=2e-6
gamma=2.6e-5

f = 369.99 ## frequency of the note
fps = int(1/dt)+1
Nt = fps * 5 ## length of one note in s
pick = 0.01 * 1 ## pick offset in cm
c = 2*L*f

#args = [c, Nt, Nx, dt, dx, l, gamma, pick]

In [146]:
ya = np.linspace(0, pick, 70)
yb = np.linspace(pick, 0, 31)
y0 = np.concatenate([ya, yb])

In [147]:
sol = np.zeros((int(Nt)+1, Nx))
Nts = np.arange(Nt)
cs =  c * (1 + 0.01*np.sin(2*np.pi* Nts / (0.05*Nt))*Nts/Nt)

In [148]:
sol[0] = y0
sol[1] = y0

In [149]:
@numba.jit("f8[:,:](f8[:,:], f8[:], i8, i8, f8, f8, f8, f8)", nopython=True, nogil=True)
def compute_cd(d, cs, times, length, dt, dx, l, gamma):
    for t in range(1, times-1):
        for i in range(2, length-2):
            outer_fact = (1/(cs[t]**2 * dt**2) + gamma/(2*dt))**(-1)
            p1 = 1/dx**2 * (d[t][i-1] - 2*d[t][i] + d[t][i+1])
            p2 = 1/(cs[t]**2 * dt**2) * (d[t-1][i] - 2*d[t][i])
            p3 = gamma/(2*dt) * d[t-1][i]
            p4 = l**2 / dx**4 * (d[t][i+2] - 4*d[t][i+1] + 6*d[t][i] - 4*d[t][i-1] + d[t][i-2])
            d[t+1][i] = outer_fact * (p1 - p2 + p3 - p4)
    return d

In [150]:
sol_cs = compute_cd(sol, cs, Nt, Nx, dt, dx, l, gamma)

In [151]:
def get_integral_fast(n):
    sin_arr = np.sin(n*np.pi*np.linspace(0,1,101))
    return np.multiply(sol, sin_arr).sum(axis=1)

In [152]:
hms = [get_integral_fast(n) for n in range(10)]

In [153]:
all_harmonics=False
if all_harmonics:
    tot = sol.sum(axis=1)[::10] # all harmonics
else:
    tot = sum(hms)[::10] # only first 10 harmonics
tot = tot.astype(np.float32)

In [154]:
#dist_tot = tot *np.exp(3*np.abs(tot)/np.abs(tot).max())
#plt.plot(dist_tot)
#plt.plot(tot)

In [155]:
wavfile.write('fis2.wav',20000,tot)
#wavfile.write('a.wav',20000,dist_tot)

class NoteGenerator():

    def __init__(self, f, vibrato, args):
        self.args = args
        self.grid = np.zeros((self.args[1], self.args[2]))
        self.grid[0:1] = np.concatenate([np.linspace(0, self.args[7], 70), np.linspace(self.args[7], 0, 31)])
        self.Nts = np.arange(self.args[1])
        self.cs = self.args[0] * (1 + 0.01*np.sin(2*np.pi* self.Nts / (0.05*self.args[1]))*self.Nts/self.args[1])

    @numba.jit("f8[:,:](f8[:,:], f8[:], i8, i8, f8, f8, f8, f8)", nopython=True, nogil=True)
    def compute_cd(self.grid, self.cs, self.args[1], self.args[2], self.args[3], self.args[4], self.args[5], self.args[6]):
        for t in range(1, times-1):
            for i in range(2, length-2):
                outer_fact = (1/(cs[t]**2 * dt**2) + gamma/(2*dt))**(-1)
                p1 = 1/dx**2 * (d[t][i-1] - 2*d[t][i] + d[t][i+1])
                p2 = 1/(cs[t]**2 * dt**2) * (d[t-1][i] - 2*d[t][i])
                p3 = gamma/(2*dt) * d[t-1][i]
                p4 = l**2 / dx**4 * (d[t][i+2] - 4*d[t][i+1] + 6*d[t][i] - 4*d[t][i-1] + d[t][i-2])
                d[t+1][i] = outer_fact * (p1 - p2 + p3 - p4)
        return d

In [156]:
e1 = wavfile.read('e1.wav')[1]
gis1 = wavfile.read('gis1.wav')[1]
h1 = wavfile.read('h1.wav')[1]
wavfile.write('E_Chord.wav', 20000, e1+gis1+h1)

In [157]:
g1 = wavfile.read('g1.wav')[1]
h1 = wavfile.read('h1.wav')[1]
d2 = wavfile.read('d2.wav')[1]
wavfile.write('G_Chord.wav', 20000, g1+h1+d2)

In [158]:
a1 = wavfile.read('a1.wav')[1]
cis2 = wavfile.read('cis2.wav')[1][:100000]
e2 = wavfile.read('e2.wav')[1][:100000]
wavfile.write('A_Chord.wav', 20000, a1+cis2+e2)

In [162]:
AChord = wavfile.read('A_Chord.wav')[1][:50000]
EChord = wavfile.read('E_Chord.wav')[1][:50000]
GChord = wavfile.read('G_Chord.wav')[1][:50000]

In [163]:
E = np.concatenate([np.zeros(10000), EChord, np.zeros(70000)])
G = np.concatenate([np.zeros(20000), GChord, np.zeros(60000)])
A = np.concatenate([np.zeros(30000), AChord, np.zeros(50000)])

E2 = np.concatenate([np.zeros(50000), EChord, np.zeros(30000)])
G2 = np.concatenate([np.zeros(60000), EChord, np.zeros(20000)])
FIS2 = np.concatenate([np.zeros(65000), wavfile.read('fis2.wav')[1][:50000], np.zeros(15000)])
A2 = np.concatenate([np.zeros(70000), AChord, np.zeros(10000)])

music = E + G + A + E2 + G2 + FIS2 + A2

wavfile.write('Smoke.wav', 20000, music)

In [ ]:
EChord

array([ 9.0865898e-01,  9.0455729e-01,  8.9535248e-01, ...,
       -5.1154487e-04,  5.2076241e-04,  1.5483816e-03], dtype=float32)